In [ ]:
pip install transformers datasets torch scikit-learn

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 1.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 8.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 10.6 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [ ]:
import torch
from transformers import T5ForConditionalGeneration, T5Tokenizer
from datasets import load_dataset
from sklearn.model_selection import train_test_split

# Load the CoNLL-2003 dataset
def load_and_preprocess_data():
    dataset = load_dataset("conll2003")
    sentences = []
    entity_sequences = []

    for split in ["train", "validation", "test"]:
        for record in dataset[split]:
            tokens = record["tokens"]
            ner_tags = record["ner_tags"]
            tags = [dataset["train"].features["ner_tags"].feature.int2str(tag) for tag in ner_tags]

            # Construct sentence and entity sequence
            sentence = " ".join(tokens)
            entity_seq = []
            for token, tag in zip(tokens, tags):
                if tag != "O":  # Extract only named entities
                    entity_seq.append(f"{token} <sep> {tag}")
            entity_sequences.append(" <sep> ".join(entity_seq))
            sentences.append(sentence)

    return sentences, entity_sequences

# Split data into train and test sets
def split_data(sentences, entity_sequences, test_size=0.2):
    return train_test_split(sentences, entity_sequences, test_size=test_size, random_state=42)

# Initialize T5 Model and Tokenizer
def initialize_model_and_tokenizer():
    tokenizer = T5Tokenizer.from_pretrained("t5-small")
    model = T5ForConditionalGeneration.from_pretrained("t5-small")
    return tokenizer, model

# Prepare data for training
def prepare_data(sentences, entity_sequences, tokenizer, max_length=512):
    inputs = tokenizer(sentences, truncation=True, padding=True, max_length=max_length, return_tensors="pt")
    outputs = tokenizer(entity_sequences, truncation=True, padding=True, max_length=max_length, return_tensors="pt")
    return inputs.input_ids, inputs.attention_mask, outputs.input_ids

# Training loop
def train_model(model, tokenizer, train_sentences, train_entities, val_sentences, val_entities, epochs=3, batch_size=8, learning_rate=5e-5):
    optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)

    train_inputs, train_masks, train_labels = prepare_data(train_sentences, train_entities, tokenizer)
    val_inputs, val_masks, val_labels = prepare_data(val_sentences, val_entities, tokenizer)

    train_dataset = torch.utils.data.TensorDataset(train_inputs, train_masks, train_labels)
    val_dataset = torch.utils.data.TensorDataset(val_inputs, val_masks, val_labels)
    train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    val_loader = torch.utils.data.DataLoader(val_dataset, batch_size=batch_size)

    for epoch in range(epochs):
        model.train()
        total_loss = 0
        for batch in train_loader:
            input_ids, attention_mask, labels = (b.to(device) for b in batch)
            optimizer.zero_grad()
            outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
            loss = outputs.loss
            loss.backward()
            optimizer.step()
            total_loss += loss.item()

        print(f"Epoch {epoch + 1}/{epochs}, Training Loss: {total_loss / len(train_loader)}")
        evaluate_model(model, tokenizer, val_loader, device)

    return model

# Evaluation
def evaluate_model(model, tokenizer, val_loader, device):
    model.eval()
    predictions, targets = [], []
    with torch.no_grad():
        for batch in val_loader:
            input_ids, attention_mask, labels = (b.to(device) for b in batch)
            outputs = model.generate(input_ids=input_ids, attention_mask=attention_mask)
            predictions.extend(tokenizer.batch_decode(outputs, skip_special_tokens=True))
            targets.extend(tokenizer.batch_decode(labels, skip_special_tokens=True))

    for i in range(5):  # Display a few examples
        print(f"Prediction: {predictions[i]}")
        print(f"Target: {targets[i]}\n")

# Main function
def main():
    sentences, entity_sequences = load_and_preprocess_data()
    train_sentences, val_sentences, train_entities, val_entities = split_data(sentences, entity_sequences)

    tokenizer, model = initialize_model_and_tokenizer()
    trained_model = train_model(
        model, tokenizer, train_sentences, train_entities, val_sentences, val_entities
    )

if __name__ == "__main__":
    main()

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/12.3k [00:00<?, ?B/s]

conll2003.py:   0%|          | 0.00/9.57k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/14041 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/3250 [00:00<?, ? examples/s]

...

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/242M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

Passing a tuple of `past_key_values` is deprecated and will be removed in Transformers v4.48.0. You should pass an instance of `EncoderDecoderCache` instead, e.g. `past_key_values=EncoderDecoderCache.from_legacy_cache(past_key_values)`.
